# Spindle Masking

In [1]:
import csv

import numpy as np
import matplotlib.pyplot as plt
import pytest

from openseize import producer

%matplotlib notebook

In [33]:
# open a csv file and column 1 of each row into a numpy array
with open('data/KJ0206_P049_Ch1-2-4_250_Hz_sleep_states.csv') as infile:
    reader = csv.reader(infile, delimiter=',')
    spindle = np.array([row[1] for row in reader])

In [34]:
# print the unique states and the number of 1-s windows SPINDLE predicted on
print('spindle contains {} states and has length {}'.format(np.unique(spindle), len(spindle)))

spindle contains ['n' 'r' 'w'] states and has length 67533


In [35]:
spindle

array(['w', 'w', 'w', ..., 'w', 'w', 'w'], dtype='<U1')

In [4]:
# Build a boolean the same length as the spindle array but True only where rem is occurring
rem_bool = spindle == 'r'
rem_idxs = np.flatnonzero(spindle=='r')
print(rem_idxs)

[ 3082  3083  3084 ... 67484 67485 67486]


In [5]:
# Build a boolean the same length as the spindle array but True only where non-rem is occurring
nrem_bool = spindle == 'n'
nrem_idxs = np.flatnonzero(nrem_bool)
print(nrem_idxs)

[  210   211   212 ... 67519 67520 67521]


In [6]:
# Here is the important bit...
# use diff to locate where mouse enters and leaves rem
# these sections will be used place Trues into a boolean mask
rem_sections = np.flatnonzero(np.diff(rem_bool)).reshape(-1,2) + 1
print(rem_sections[1:20])

[[ 3200  3205]
 [ 5160  5168]
 [ 5348  5356]
 [ 6088  6105]
 [ 7047  7080]
 [ 7188  7195]
 [ 7287  7304]
 [ 7700  7710]
 [ 7783  7800]
 [12400 12426]
 [12749 12769]
 [13000 13006]
 [14107 14113]
 [14336 14373]
 [14500 14529]
 [14879 14898]
 [17847 17868]
 [18060 18070]
 [18852 18866]]


In [7]:
# again build sections for non-rem
nrem_sections = np.flatnonzero(np.diff(nrem_bool)).reshape(-1,2) + 1
print(nrem_sections)

[[  210   214]
 [  216   229]
 [  233   239]
 ...
 [67495 67496]
 [67503 67517]
 [67519 67522]]


**IMPORTANT We have assumed that the mouse always enters and leaves a sleep state, that is, we have assumed an even number of transitions (2 for every time the animal enters/leaves a specific state). This is wrong if the animal starts or ends in the sleep state.**


In [17]:
def spindle_array(fname, column=1, **kwargs):
    """Reads a SPINDLE sleep classification file to a 1-D numpy array.
    
    Args:
        fname: str
            A string filename or Path instance to a SPINDLE csv file.
        column: int
            The column containing the sleep class.
        kwargs:
            Any valid keyword argument for pythons csv reader builtin.
            
    Returns:
        A 1-D array containing state identifiers.
    """
    
    with open(fname, 'r', newline='') as infile:
        reader = csv.reader(infile, **kwargs)
        spindle = np.array([row[column] for row in reader])
    
    return spindle

In [36]:
def spindle_mask(arr, states, fs, unit=1):
    """Reads a spindle file into a 1-D numpy boolean array for masking a producer.
    
    Args:
        arr: 1-D array
            An array containing the spindle classes.
        states: sequence
            A list of states in the spindle file that survive the mask.
        fs: int
            The sampling rate of the recording.
        unit: float
            The time difference in seconds between successive rows in the spindle file.
            Defaults to 1 second time intervals.
            
    Returns:
        A 1-D boolean array delineating which samples will be masked.
    """
    
    # compute which windows of unit length are True in the mask
    window_submask = [arr == state for state in states]
    window_mask = np.logical_or.reduce(window_submask)
    
    # set first & last window to False ensuring number of transistions is always even
    window_mask[0] = False
    window_mask[-1] = False
    
    # compute transitions and pair them along new axis
    transitions = np.diff(window_mask)
    sections = np.flatnonzero(transitions).reshape(-1,2) + 1
    
    # build a sample mask
    mask = np.zeros((len(arr)*fs*unit))
    for section in sections:
        samples = section * fs * unit
        mask[slice(*samples)] = True
    
    return sections, mask

In [37]:
# read spindle to an array of classes
fname = 'data/KJ0206_P049_Ch1-2-4_250_Hz_sleep_states.csv'
arr = spindle_array(fname, delimiter=',')

# build a mask
sections, mask = spindle_mask(arr, states=['n','r'], fs=250, unit=4)
print(sections[0:20])

[[ 210  214]
 [ 216  229]
 [ 233  239]
 [ 241  301]
 [ 314  320]
 [ 321  353]
 [ 356  390]
 [ 396  406]
 [ 409  423]
 [ 428  431]
 [ 437  482]
 [ 492  494]
 [ 495  503]
 [ 508  538]
 [ 540  555]
 [1322 1458]
 [1460 1463]
 [1464 1465]
 [1471 1497]
 [1499 1569]]


In [38]:
print(np.flatnonzero(mask))

[  210000   210001   210002 ... 67521997 67521998 67521999]


In [28]:
# testing

arr = np.array(['w']*100)

# add a set of arbitrary states to arr
states = [('r', slice(11,16)), ('r', slice(30, 44)), ('n', slice(66, 67)), ('n', slice(90, 94))]
for state, sl in states:
    arr[sl] = state

In [29]:
arr

array(['w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'r', 'r',
       'r', 'r', 'r', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w',
       'w', 'w', 'w', 'w', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r',
       'r', 'r', 'r', 'r', 'r', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w',
       'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w',
       'w', 'n', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w',
       'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'n',
       'n', 'n', 'n', 'w', 'w', 'w', 'w', 'w', 'w'], dtype='<U1')

In [30]:
sections, mask = spindle_mask(arr, states=['r'], fs=1)

In [31]:
sections

array([[11, 16],
       [30, 44]])

In [32]:
mask

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])